In [20]:
import geopandas as gpd
import hashlib
import rasterio
from rasterio.enums import Resampling
from rasterio.windows import from_bounds
import os
from rasterio.warp import calculate_default_transform, reproject, Resampling
from shapely.geometry import box
import numpy as np
import re
import shutil
from shapely.geometry import Point

## Resample and Crop L-band to 224x224 Using Grid

In [27]:
with rasterio.open("./rice_practices/SAOCOM/EOL1ASARSAO1A11592909_2024-12-25_geo.tif") as src:
    print("Resolution:", src.res)  # (xres, yres)
    print("CRS:", src.crs)


Resolution: (0.00019989685404847444, 0.00019995554567681697)
CRS: EPSG:4326


In [37]:
target_crs = "EPSG:32644"
tmp_dir = "./rice_practices/SAOCOM_chips_reproj"
os.makedirs(tmp_dir, exist_ok=True)

In [38]:
lband_folder = "./rice_practices/SAOCOM"
output_folder = "./rice_practices/SAOCOM_chips_10_meter_new"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Extract date from filename
def extract_date(filename):
    match = re.search(r'_(\d{4}-\d{2}-\d{2})_', filename)
    if match:
        return match.group(1)
    match = re.search(r'_(\d{4}-\d{2}-\d{2})\.tif$', filename)
    if match:
        return match.group(1)
    return "unknown"

# Loop over all grids
for _, row in grids.iterrows():
    fid = row["FID"]
    bounds = row.geometry.bounds
    print(f"\nProcessing grid: {fid}")

    for fname in os.listdir(lband_folder):
        if not fname.endswith(".tif"):
            continue
        date_str = extract_date(fname)
        fpath = os.path.join(lband_folder, fname)

        # Reproject L-band to UTM (EPSG:32644)
        with rasterio.open(fpath) as src:
            dst_transform, dst_width, dst_height = calculate_default_transform(
                src.crs, target_crs, src.width, src.height, *src.bounds, resolution=10
            )
            kwargs = src.meta.copy()
            kwargs.update({
                "crs": target_crs,
                "transform": dst_transform,
                "width": dst_width,
                "height": dst_height
            })

            tmp_path = os.path.join(tmp_dir, f"reprojected_{os.path.splitext(fname)[0]}.tif")
            with rasterio.open(tmp_path, "w", **kwargs) as dst:
                for b in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, b),
                        destination=rasterio.band(dst, b),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=dst_transform,
                        dst_crs=target_crs,
                        resampling=Resampling.bilinear
                    )

        with rasterio.open(tmp_path) as reproj:
            chip_geom = box(*reproj.bounds)

            # Check if this L-band image intersects this grid
            if not row.geometry.intersects(chip_geom):
                continue

            # Crop to grid bounds
            crop_window = from_bounds(*bounds, transform=reproj.transform)
            try:
                cropped = reproj.read(window=crop_window)
            except:
                print(f"Failed to crop {fname} for grid {fid}")
                continue

            transform = rasterio.windows.transform(crop_window, reproj.transform)

            out_name = f"{fid}_{date_str}_lband.tif"
            out_path = os.path.join(output_folder, out_name)

            # Total pixel checks
            total_pixels = np.prod(cropped.shape)
            nan_ratio = np.isnan(cropped).sum() / total_pixels
            zero_ratio = (cropped == 0).sum() / total_pixels

            if nan_ratio > 0.05:
                print(f"skipped >5% NaN: {out_name} ({nan_ratio:.2%} NaNs)")
                continue

            if zero_ratio > 0.05:
                print(f"skipped >5% zeros: {out_name} ({zero_ratio:.2%} zeros)")
                continue

            with rasterio.open(
                out_path, "w",
                driver="GTiff",
                height=224,
                width=224,
                count=reproj.count,
                dtype=reproj.dtypes[0],
                crs=reproj.crs,
                transform=transform
            ) as dst:
                dst.write(cropped)

            print(f"Saved: {out_name}")
